__собираем предложения о земельных участках__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/land/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_land.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/land/avito/avito_2022-09-08_16-05_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-12_13-51_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-13_13-56_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-15_10-31_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-16_15-10_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-19_12-39_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-22_09-57_raw.xlsx']

In [7]:
# for f in raw_data_files:
#     df = pd.read_excel(f)
#     df['place']='sevastopol'
#     df.to_excel(f,index=False)
#     print(f)

In [8]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

7728


In [9]:
# data_avito

In [10]:
from lib.avito import AvitoDataCleanerRealtyLand

data_avito = AvitoDataCleanerRealtyLand().transform( data_avito )

print(len(data_avito))

7728


In [11]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,area,is_IJS,priceM,priceMU,area_size_category
5814,2264660271,Участок 5 сот. (ИЖС),2495000,,Нахимовский муниципальный округ,"Хороший видовой участок, ижс, звоните все расскажу) по факту 8 соток,",6,2022-09-19 12:39:53.571,sevastopol,5.00,True,2.50,0.50,4-8
1196,2536936493,Участок 5 сот. (ИЖС),2500000,,ул. Александра Земкова,"Продаю участок 5 соток в Севастополе в Балаклавском районе по улице Александра Земкова. \n\nКатегория: земли населенных пунктов, для строительства и обслуживания жилого дома, хозяйственных построек и сооружений. \n\nГаз — непосредственно на границе участка, во",3,2022-09-12 13:51:16.745,sevastopol,5.00,True,2.50,0.50,4-8
3681,890631863,"Участок 6,6 сот. (ИЖС)",10000000,,садовое товарищество Рыбак-2Б,"Продается участок Ижс в Севастополе на проспекте Столетовском, район ""Летчики"" — Тсн ""Жстиз ""Столетовский"" (бывший Рыбак-2). Точный адрес — ул. Январская, 34. \nКадастровый номер: 91:02:001007:943.\nВ собственности. Документы Российские. \nУчасток огорожен, в",8,2022-09-15 10:31:26.039,sevastopol,6.60,True,10.00,1.52,4-8


In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728 entries, 0 to 7727
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            7728 non-null   int64         
 1   title               7728 non-null   object        
 2   price               7728 non-null   int64         
 3   obj_name            7728 non-null   object        
 4   adr                 7728 non-null   object        
 5   description         7728 non-null   object        
 6   avito_page          7728 non-null   int64         
 7   ts                  7728 non-null   datetime64[ns]
 8   place               7728 non-null   object        
 9   area                7728 non-null   float64       
 10  is_IJS              7728 non-null   bool          
 11  priceM              7728 non-null   float64       
 12  priceMU             7728 non-null   float64       
 13  area_size_category  7728 non-null   category    

---

In [14]:
data_avito['url'] = 'https://avito.ru/' + data_avito['avito_id'].astype(int).astype(str)
data_avito = data_avito.drop(columns=['avito_id'])
# data_cian = data_cian.rename(columns={'cian_url':'url'})

In [15]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

7728


## обновляем таблицу адресов

In [16]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

In [17]:
places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [18]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

  0%|          | 0/7728 [00:00<?, ?it/s]

  0%|          | 0/7728 [00:00<?, ?it/s]

In [19]:
data[['adr_orig','adr']].sample(10)

,adr_orig,adr
7263,"ул. Абазы, 30","Севастополь,ул. Абазы, 30"
7292,Редутная ул.,"Севастополь,Редутная ул."
7606,район Северная Сторона,"Севастополь,район Северная Сторона"
380,садовое товарищество Ригель,"Севастополь,садовое товарищество Ригель"
6654,"ул. Горпищенко, 151А","Севастополь,ул. Горпищенко, 151А"
4759,"садоводческое товарищество Берег, 248","Севастополь,садоводческое товарищество Берег, 248"
1273,"садоводческое товарищество Парус-5, 24","Севастополь,садоводческое товарищество Парус-5, 24"
4113,"с. Песчаное, Речная ул.","Севастополь,с. Песчаное, Речная ул."
6807,"с. Береговое, Магистральная ул.","Севастополь,с. Береговое, Магистральная ул."
4694,"с. Штурмовое, Шулинская ул., 25","Севастополь,с. Штурмовое, Шулинская ул., 25"


In [20]:
# from lib.locator import GeocoderSimpleOSM
from lib.locator import GeocoderYandex
from yandex_keys import keys

atr = AddressTransformer()
locator = GeocoderYandex(keys[0])
# locator = GeocoderSimpleOSM()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

if len(loc_undef)>0:
    # определяем координаты по строке адреса
    loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
    logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')
    loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 
    assert len(loc)>0
    loc.to_pickle(loc_file_path)

[INFO    ] 2022-09-22 11:53:59 | LocationUpdater: 6042 addresses in index
[INFO    ] 2022-09-22 11:53:59 | LocationUpdater: 6042 addresses total
[INFO    ] 2022-09-22 11:54:00 | LocationUpdater: 6042 addresses defined
[INFO    ] 2022-09-22 11:54:00 | LocationUpdater: 0 addresses undefined


In [21]:
print( len(loc) )
loc.sample(3)

6042


,latitude,longitude,adr
3657,44.577882,33.534262,"Севастополь,ул. Лётчика Мусатова, 34"
5193,44.52028,33.508005,"Севастополь,садоводческое товарищество Надежда-1, Ковыльный пер."
4841,44.738953,33.552615,"Севастополь,СОТ Южный, 265"


## дополняем данные геометкой

In [22]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

assert len(data)>0
data.to_pickle(result_file_path)

data.info()

всего записей: 7728
записей без геометки: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7728 entries, 0 to 7727
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               7728 non-null   object        
 1   price               7728 non-null   int64         
 2   obj_name            7728 non-null   object        
 3   adr_orig            7728 non-null   object        
 4   description         7728 non-null   object        
 5   avito_page          7728 non-null   int64         
 6   ts                  7728 non-null   datetime64[ns]
 7   place               7728 non-null   object        
 8   area                7728 non-null   float64       
 9   is_IJS              7728 non-null   bool          
 10  priceM              7728 non-null   float64       
 11  priceMU             7728 non-null   float64       
 12  area_size_category  7728 non-null   category      
 13  url 